In [3]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.0MB 9.2MB/s 
     |████████████████████████████████| 890kB 44.4MB/s 
     |████████████████████████████████| 3.2MB 39.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=0f23c00b3f893036ad06ca7d2ac4ac8b6235625dfeb0bfe357da0c0bba5292d1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 7.5MB/s 


In [4]:
from google.colab import drive
drive.mount('/content/drive')
root_folder = "/content/drive/My Drive/cs182_final_project/cs182-nlp (master)"

Mounted at /content/drive


In [12]:
import os
import sys
sys.path.append(root_folder)

import json
import torch
import pandas as pd

In [10]:
# result = [json.loads(jline) for jline in response.read().splitlines()]

json_file_name = "yelp_review_training_dataset.jsonl"

if False:
  print("Using Sample Eval jsonl smaller file")
  json_file_name = "sample_eval.jsonl"

yelp_ratings_jsonl = []

with open(root_folder + "/" + json_file_name, "r") as f:
  for line in f:
    yelp_ratings_jsonl.append(json.loads(line))

print(yelp_ratings_jsonl[:10])

[{'review_id': 'Q1sbwvVQXV2734tPgoKj4Q', 'text': 'Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.', 'stars': 1.0}, {'review_id': 'GJXCdrto3ASJOqKeVWPi6Q', 'text': "I *adore* Travis at the Hard Rock's new Kelly Cardenas Salon!  I'm always a fan of a great blowout and no stranger to the chains that offer this service; however, Travis has taken the flawless blowout to a whole new level!  \n\nTravis's greets you with his perfectly green swoosh in his otherwise perfectly styled black hair and a Vegas-worthy rockstar outfit.  Next comes the most relaxing and incredible shampoo -- where you get a full head message that could cure even the very worst migraine in minutes --- and the scented shampoo room.  Travis has freakishly strong fingers (in a good way) and use the perfect amount of pressure.  That was superb!  Then starts the glorious 

In [13]:
yelp_ratings = pd.DataFrame.from_records(yelp_ratings_jsonl)
display(yelp_ratings)

,review_id,text,stars
0,Q1sbwvVQXV2734tPgoKj4Q,Total bill for this horrible service? Over $8G...,1.0
1,GJXCdrto3ASJOqKeVWPi6Q,I *adore* Travis at the Hard Rock's new Kelly ...,5.0
2,2TzJjDVDEuAW6MR5Vuc1ug,I have to say that this office really has it t...,5.0
3,yi0R0Ugj_xUx_Nek0-_Qig,Went in for a lunch. Steak sandwich was delici...,5.0
4,11a8sVPMUFtaC7_ABRkmtw,Today was my second out of three sessions I ha...,1.0
...,...,...,...
533576,2vQO_kmSr6YPBrR8GH_FPA,Dr Young and her assistants take obvious pride...,5.0
533577,DUdLTGVpgsi0sv_g4A5ITQ,We started our 20 month of daughter here on an...,5.0
533578,AKGELpRNTTXajuZHbPxdJg,"First of all, they are supposed to open at 9:0...",2.0
533579,ghYZM7lqzjej05I_T3vYyA,It's not often that you visit a company and th...,5.0


In [15]:
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'xlnet-base-cased')


Downloading: "https://github.com/huggingface/pytorch-transformers/archive/master.zip" to /root/.cache/torch/hub/master.zip


In [16]:
print(model.forward(yelp_ratings["text"]))

TypeError: ignored